**Introduction**

New york is the most populous city in the United States,but is so expensive for a lot of tourist,to enjoy a cheaper lunch and save money, most tourists prefer to buy pizza. So to start and succeed this project must be resolved, and that is to analyze the Pizza stores’ locations in the major US cities and find the best place for our tourist . Our main target are tourists with a taste of western style pizza.


**Data section**

I will use the FourSquare API to collect data about locations of Pizza stores in 5 major US cities which are: New York,NY, San Francisco, CA, Jersey City, NJ, Boston, MA and Chicago,IL. These are one of the most populated US cities and I am hopeful that they will contain the best Pizza places in the US.

**Methodology**

our main target here is to asses which city would have the highest Pizza store density , we participated in using the Four Square API through the venues channel.In addition we took advantage of near query to get venues cities.Also with the CategoryID purpose to set it to show only pizza places.An examples of my requests following with the help of this link: https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=20180605&New York, NY&limit=100&categoryId=4bf58dd8d48988d1ca941735

That 4bf58dd8d48988d1ca941735 is the Id of the Pizza Place Category. Also, Foursquare limits us to maximum of 100 venues per query.

Moreover, I repeated this request for the 5 studied cities and got their top 100 venues. I saved the name and coordinate data only from the result and plotted them on the map for visual inspection.

Next, to get an indicator of the density of Pizza Places, I calculated a center coordinate of the venues to get the mean longitude and latitude values. Then I calculated the mean of the Euclidean distance from each venue to the mean coordinates. That was my indicator; mean distance to the mean coordinate.


In [2]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize 
import folium 

In [3]:
CLIENT_ID = '2Y3IB5KEL5ER4ZUOFMSJLR1ABIHBEKYW2XWUXN5NSY1Y0CCH' 
CLIENT_SECRET = 'OD4GQHVFETRL0V5OCMV4EVVPQMMZP0LZ50UCXQ45AKMN051V'
VERSION = '20180605'

In [4]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") 
    results[city] = requests.get(url).json()

In [5]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [6]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in New York, NY =  291
Showing Top 100
Total number of pizza places in Chicago, IL =  225
Showing Top 100
Total number of pizza places in San Francisco, CA =  168
Showing Top 100
Total number of pizza places in Jersey City, NJ =  125
Showing Top 100
Total number of pizza places in Boston, MA =  188
Showing Top 100


**Some maps that we will use to realise the project**

In [7]:
maps[cities[0]]